# Introduction

We want our model to estimate election outcomes based on the fundamentals and poll data. Thus, we need a "poll feature" for each district upon which to train. Unfortunately, many districts do not have poll data. We overcome this by interpolating poll results for districts that do not have polls from districts that do have polls based on districts' similarity in demographics and historical election results.

The poll data points upon which we will interpolate are [538](https://fivethirtyeight.com/features/2018-house-forecast-methodology/)'s adjusted poll margins. We will pull the most recent adjusted margin (MRAM).

We perform the interpolation using [radial basis function interpolation](http://num.math.uni-goettingen.de/schaback/teaching/sc.pdf) (RBF interpolation).

In [7]:
# !pip install git+git://github.com/treverhines/RBF@master
    
import pandas as pd
import numpy as np
import scipy.cluster
import scipy.interpolate
from scipy import stats
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
from sklearn.decomposition import FastICA
# from rbf.interpolate import RBFInterpolant
# import rbf.basis

from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

import matplotlib.pyplot as plt
plt.style.use('ggplot')

print('done')

done


# Input Data

We will now interpolate real polls based on a large range of input features. We predict based on ProximityOne's [demographic data by congressional district](http://proximityone.com/cd.htm). Specifically, we input the [social characteristics](http://proximityone.com/cd161dp2.htm).

In [8]:
df = pd.read_excel('all.xlsx', sheet='Sheet 1')

ins = ['S' + str(rep).zfill(3) for rep in range(1, df.shape[1])]
outs = ['MRAM']

print('done')

done


# Preprocessing

We obviously must drop all non-numeric data. However, we also must drop rows in which coordinates are duplicated due to quirks in the way the RBF interpolation is calculated (duplicate coordinates result in `scipy` trying to invert a singular matrix).

In [20]:
# Drop rows that have non-numerical data
df.dropna(inplace=True)
df = df[df.applymap(np.isreal).any(1)]
df.columns = ['MRAM'] + ins

# Drop rows that down have MRAM
df = df[df['MRAM'] != 999]

# tidx = [np.random.randint(0, len(df)) for i in range(int(5 * len(df) / 10))]
# vidx = list(set(range(515)).difference(set(tidx)))

tidx = range(0, int(len(df) / 2))
vidx = range(int(len(df) / 2), len(df))

trows = [df.iloc[idx] for idx in tidx]
vrows = [df.iloc[idx] for idx in vidx]
df_train = df.drop(df.index[tidx])
df_validate = df.drop(df.index[vidx])

ica = FastICA(n_components=50)
S_ica_ = ica.fit(df.as_matrix())
mloss = S_ica_.inverse_transform(S_ica_.fit_transform(df.as_matrix())) - df.as_matrix()  # Estimate the sources

for i in range(517):
    mloss[:,i] /= np.std(mloss[:,i])

print(mloss)

# def get_repeat_idxs(records_array):
#     idx_sort = np.argsort(records_array)
#     sorted_records_array = records_array[idx_sort]
#     vals, idx_start, count = np.unique(sorted_records_array, return_counts=True,
#                                     return_index=True)
#     res = np.split(idx_sort, idx_start[1:])
#     vals = vals[count > 1]
#     return filter(lambda x: x.size > 1, res)

# # Drop data columns that have repeats
# bads = []
# for cni in range(len(ins)):
#     cn = ins[cni]
#     col = np.array(df[cn])
#     repeated = list(get_repeat_idxs(col))
    
#     if len(repeated) > 0:
#         bads.append(cn)

# for cn in bads:
#     ins.remove(cn)
#     df.drop(cn, axis=1, inplace=True)
    
# # Drop rows with repeats
# for name in ins + outs:
#     _, uidx = np.unique(df[name], return_index=True)
#     df = df.iloc[uidx]

# df.reset_index(drop=True, inplace=True)
# df.to_csv('tmp.csv', sep=',')

# print('done')

SyntaxError: invalid syntax (<ipython-input-20-86207e0f1518>, line 25)

# Training

In [4]:
def gen_interpolator(df_train, _ins, _outs):
    inrep = [np.array(df_train[rep]).astype(float) for rep in _ins]
    outrep = [np.array(df_train[rep]).astype(float) for rep in _outs]
    features = list(inrep) + list(outrep)
    
    # Check for repeats (there shouldn't be any)
    for feature in features:
        if len(feature) != len(set(feature)):
            print('Repeat')
    return scipy.interpolate.Rbf(*features)

# Validation

We partition our data into training and validation sets. We train the interpolation on most of the available data points, leaving out a set of validation points. We then predict the *MRAM* of the validation points and compare to the true *MRAM*, calculating the error in our interpolation.

**TODO**: get the correct number of training points corresponding to the number of available polls

In [5]:
# Validate interpolation on many partitions
totcor = 0
y_pred_all = []
y_true_all = []
N = 1
for k in range(N):
#     tidx = [np.random.randint(0, len(df)) for i in range(int(5 * len(df) / 10))]
    tidx = range(85)
    trows = [df.iloc[idx] for idx in tidx]
    df_train = df.drop(df.index[tidx])
    
    
    inrep = [np.array(df_train[rep]).astype(float) for rep in ins]
    outrep = [np.array(df_train[rep]).astype(float) for rep in outs]
    features = list(inrep) + list(outrep)
    
    # Check for repeats (there shouldn't be any)
    for feature in features:
        if len(feature) != len(set(feature)):
            print('Repeat')
            
            
            
    rbfi = scipy.interpolate.Rbf(*features)
    
    
    
    
    
#     rbfi = gen_interpolator(df_train, ins, outs)
    
    y_pred = [float(rbfi(*[row[rep] for rep in ins])) for row in trows]
    y_true = [row[outs[0]] for row in trows]

    correct = []
    for i in range(len(y_pred)):
        correct.append((y_pred[i] > 0.5) == (y_true[i] > 0.5))

    totcor += sum(correct) / len(y_pred)
    y_pred_all.extend(y_pred)
    y_true_all.extend(correct)

# This represents the expected rate at which the interpolator correctly predicts the outcome of the race
print('Fraction of Races Predicted Correctly: ' + str(totcor / N))
print('Mean Squared Prediction Error: ' + str(mse(y_true_all, y_pred_all)))
print('Mean Absolute Prediction Error: ' + str(mae(y_true_all, y_pred_all)))
print('Mean Prediction Error: ' + str(np.mean(np.array(y_true_all) - np.array(y_pred_all))))
print('Stdev Prediction Error: ' + str(np.std(np.array(y_true_all) - np.array(y_pred_all))))

plt.hist(np.array(y_true_all) - np.array(y_pred_all))
plt.title('y_true - y_pred')
plt.show()

print('done')

Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat
Repeat


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


LinAlgError: Matrix is singular.

In [ ]:
res = stats.probplot(np.array(y_true_all) - np.array(y_pred_all), plot=plt)

print('done')

# Final Answer

We now plug in all available data to an interpolator and interpolate all rows. Keep in mind we expect our interpolation error to be drawn from the distribution shown above.

In [ ]:
# TODO do it